# Interstellar : Calcul de l'âge moyen des acteurs et actrices

## Introduction

Dans ce notebook, nous allons calculer l'âge moyen des acteurs et actrices du film **Interstellar** en utilisant le web crawling.

Nous suivrons les étapes suivantes :

1. **Objectif du projet** : Comprendre les étapes du crawling et du traitement des données.
2. **Exploration de la base de données cible** : Extraction des informations depuis Allociné.
3. **Calcul des âges** : Analyse et calcul des résultats.

## Étape 1 : Importation des bibliothèques nécessaires

In [1]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

## Étape 2 : Analyse de la structure de la page Web

Le lien vers la filmographie de Christopher Nolan est : [https://www.allocine.fr/personne/fichepersonne-30367/filmographie](https://www.allocine.fr/personne/fichepersonne-30367/filmographie).

Nous devons :
- Identifier les informations sur les acteurs et actrices pour le film *Interstellar*.
- Extraire les liens vers les pages de profil des acteurs.

In [2]:
# Étape 2.1 : Télécharger la page Web
data_url = "https://www.allocine.fr/personne/fichepersonne-30367/filmographie"
response = requests.get(data_url)

# Vérification du statut de la requête
if response.status_code == 200:
    print("Page téléchargée avec succès !")
else:
    print("Erreur lors du téléchargement de la page.")

Page téléchargée avec succès !


In [3]:
# Étape 2.2 : Parser le contenu avec BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Étape 2.3 : Localiser la section du film Interstellar
# Extraction des informations sur les acteurs et actrices
interstellar_section = soup.find('a', string='Interstellar')

if interstellar_section:
    film_url = "https://www.allocine.fr" + interstellar_section['href']
    print(f"URL du film Interstellar : {film_url}")
else:
    print("Film Interstellar non trouvé dans la filmographie.")

URL du film Interstellar : https://www.allocine.fr/film/fichefilm_gen_cfilm=114782.html


## Étape 3 : Extraction des données des acteurs et actrices

Nous accédons maintenant à la page d'Interstellar pour extraire les profils des acteurs et actrices.

In [20]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np

# URL de la page de casting du film Interstellar
base_url = "https://www.allocine.fr"
casting_url = f"{base_url}/film/fichefilm-114782/casting/"

# Récupérer les liens des fiches acteurs
reponse = requests.get(casting_url)
soup = BeautifulSoup(reponse.text, "html.parser")

# Trouver la section contenant les acteurs
casting_section = soup.find("section", class_="section casting-actor")

if not casting_section:
    raise Exception("La section contenant le casting n'a pas été trouvée. Vérifiez la structure HTML.")

# Récupérer les liens des acteurs, en incluant toutes les classes finissant par 'link' (y compris 'xXx item link')
actor_links = []

# Cibler toutes les balises <a> qui ont un attribut href, et qui contiennent la classe 'link' (quelle que soit la structure exacte de la classe)
for link_tag in casting_section.find_all("a", href=True):
    class_attr = link_tag.get('class', [])
    # Vérifier si la classe contient un élément 'link', peu importe les autres parties de la classe
    if any('link' in class_name for class_name in class_attr):
        actor_links.append(base_url + link_tag["href"])

# Supprimer les doublons éventuels
actor_links = list(set(actor_links))  # On utilise set() pour garder les liens uniques
print(f"Nombre total d'acteurs trouvés : {len(actor_links)}")
print("Exemple de liens :")
print(actor_links[:5])  # Afficher les 5 premiers liens


Nombre total d'acteurs trouvés : 23
Exemple de liens :
['https://www.allocine.fr/personne/fichepersonne_gen_cpersonne=2535.html', 'https://www.allocine.fr/personne/fichepersonne_gen_cpersonne=525421.html', 'https://www.allocine.fr/personne/fichepersonne_gen_cpersonne=1192.html', 'https://www.allocine.fr/personne/fichepersonne_gen_cpersonne=46687.html', 'https://www.allocine.fr/personne/fichepersonne_gen_cpersonne=81745.html']


In [21]:
# DataFrame pour stocker les résultats
actor_data = []

# Parcourir chaque lien des acteurs
for actor_link in actor_links:
    # Accéder à la page de chaque acteur
    response = requests.get(actor_link)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Extraire le nom de l'acteur
    name_tag = soup.find("div", class_="titlebar-title titlebar-title-xl")
    if name_tag:
        actor_name = name_tag.get_text(strip=True)
    else:
        actor_name = "Nom inconnu"

    # Extraire la date de naissance à partir du JSON-LD
    birth_year = None
    script_tag = soup.find("script", type="application/ld+json")
    if script_tag:
        json_data = script_tag.string
        if '"birthDate":' in json_data:
            # Extraire l'année de naissance du champ birthDate
            birth_date = json_data.split('"birthDate":')[1].split('"')[1]  # Extrait la date complète
            birth_year = birth_date.split('-')[0]  # Récupère l'année (avant le premier tiret)

    # Ajouter les informations dans le DataFrame
    actor_data.append([actor_name, birth_year])

# Créer un DataFrame avec les informations
df_actors = pd.DataFrame(actor_data, columns=["Nom", "Année de Naissance"])

# Afficher le DataFrame
print(df_actors)


                       Nom Année de Naissance
0            Michael Caine               1933
1        Timothée Chalamet               1995
2               Matt Damon               1970
3             Topher Grace               1978
4      Francis X. McCarthy               None
5            Casey Affleck               1975
6            Ellen Burstyn               1932
7              Elyes Gabel               1983
8           William Devane               1937
9              Wes Bentley               1978
10        Jessica Chastain               1977
11           Anne Hathaway               1982
12           David Oyelowo               1976
13  Mark Casimir Dyniewicz               None
14            John Lithgow               1945
15          Collette Wolfe               None
16     Matthew McConaughey               1969
17            Andrew Borba               None
18            Lena Georgas               None
19           Mackenzie Foy               2000
20             Leah Cairns        

## Étape 4 : Calcul de l'Age Moyen des Acteurs

### Calcul de l'Age des Acteurs lors de la sortie du Film en 2014

In [24]:
# Conversion de la colonne "Année de Naissance" en type integer INT
df_actors['Année de Naissance'] = pd.to_numeric(df_actors['Année de Naissance'], errors='coerce', downcast='integer')

# Ajouter une colonne "Age" correspondant à l'âge en 2014
df_actors['Age'] = df_actors['Année de Naissance'].apply(lambda x: 2014 - x if pd.notnull(x) else None)

# Afficher le DataFrame
print(df_actors)

                       Nom  Année de Naissance   Age
0            Michael Caine              1933.0  81.0
1        Timothée Chalamet              1995.0  19.0
2               Matt Damon              1970.0  44.0
3             Topher Grace              1978.0  36.0
4      Francis X. McCarthy                 NaN   NaN
5            Casey Affleck              1975.0  39.0
6            Ellen Burstyn              1932.0  82.0
7              Elyes Gabel              1983.0  31.0
8           William Devane              1937.0  77.0
9              Wes Bentley              1978.0  36.0
10        Jessica Chastain              1977.0  37.0
11           Anne Hathaway              1982.0  32.0
12           David Oyelowo              1976.0  38.0
13  Mark Casimir Dyniewicz                 NaN   NaN
14            John Lithgow              1945.0  69.0
15          Collette Wolfe                 NaN   NaN
16     Matthew McConaughey              1969.0  45.0
17            Andrew Borba                 NaN

### Calcul de l'age moyen des acteurs

In [27]:
# Calculer l'âge moyen des acteurs
age_moyen = df_actors['Age'].mean()

# Afficher l'âge moyen
print(f"L'âge moyen des acteurs d'Interstellar lors de sa sortie en 2014 est : {age_moyen:.2f} ans")


L'âge moyen des acteurs d'Interstellar lors de sa sortie en 2014 est : 44.06 ans
